<a href="https://colab.research.google.com/github/palVikram/Machine-Learning-using-Python/blob/master/Language_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


## Download the data


In [2]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip


['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

## Configuration


In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"


## Prepare the data


In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


## Build the model


In [90]:
# Define an input sequence and process it.
encoder_inputs = tf.keras.Input(shape=(None, num_encoder_tokens))
encoder =tf.keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = tf.keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)


attention = tf.keras.layers.Attention(name="attention_layer")([decoder_outputs,encoder_outputs])

decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name="concat_layer")([decoder_outputs, attention])

decoder_dense = tf.keras.layers.Dense(num_decoder_tokens, activation="softmax")

decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.summary()

Model: "model_44"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_113 (InputLayer)         [(None, None, 71)]   0           []                               
                                                                                                  
 input_114 (InputLayer)         [(None, None, 93)]   0           []                               
                                                                                                  
 lstm_112 (LSTM)                [(None, 256),        335872      ['input_113[0][0]']              
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                           

## Train the model


In [116]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=200,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/200
125/125 [==============================] - 5s 22ms/step - loss: 0.0514 - accuracy: 0.9819 - val_loss: 0.9811 - val_accuracy: 0.8450
Epoch 2/200
125/125 [==============================] - 2s 15ms/step - loss: 0.0492 - accuracy: 0.9826 - val_loss: 0.9755 - val_accuracy: 0.8460
Epoch 3/200
125/125 [==============================] - 2s 15ms/step - loss: 0.0481 - accuracy: 0.9828 - val_loss: 0.9665 - val_accuracy: 0.8467
Epoch 4/200
125/125 [==============================] - 2s 14ms/step - loss: 0.0473 - accuracy: 0.9833 - val_loss: 0.9857 - val_accuracy: 0.8453
Epoch 5/200
125/125 [==============================] - 2s 14ms/step - loss: 0.0470 - accuracy: 0.9832 - val_loss: 0.9752 - val_accuracy: 0.8450
Epoch 6/200
125/125 [==============================] - 2s 14ms/step - loss: 0.0464 - accuracy: 0.9834 - val_loss: 0.9559 - val_accuracy: 0.8474
Epoch 7/200
125/125 [==============================] - 2s 14ms/step - loss: 0.0459 - accuracy: 0.9834 - val_loss: 0.9654 - val_accuracy:

## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [117]:
model.layers

In [118]:
model = keras.models.load_model("s2s")

# Get encoder inputs, outputs and states
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]

# Create encoder model
encoder_model = keras.Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_65"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_113 (InputLayer)      [(None, None, 71)]        0         
                                                                 
 lstm_112 (LSTM)             [(None, 256),             335872    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 335,872
Trainable params: 335,872
Non-trainable params: 0
_________________________________________________________________


In [119]:

model = keras.models.load_model("s2s")

# Get encoder inputs, outputs and states
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]

# Create encoder model
encoder_model = tf.keras.Model(inputs=[encoder_inputs],outputs=[encoder_states,encoder_outputs])


# Get decoder inputs, outputs and states
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]

decoder_input2 = keras.Input(shape=(None,256))

attention = model.get_layer('attention_layer')([decoder_outputs,decoder_input2])

# Concatenate attention and decoder outputs
decoder_outputs = model.layers[5]([decoder_outputs, attention])

decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)

# Create decoder model
decoder_model = keras.Model(
    [decoder_inputs,decoder_input2, decoder_states_inputs], [decoder_outputs] + decoder_states
)


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

You can now generate decoded sentences as such:


In [120]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value,encoder_outputs = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq, encoder_outputs,states_value])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [121]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 339ms/step


1/1 [==============================] - 0s 16ms/step
-
Input sentence: Go.
Decoded sentence: Va-lle!

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Go.
Decoded sentence: Va-lle!

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Go.
Decoded sentence: Va-lle!

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Go.
Decoded sentence: Va-lle!

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Hi.
Decoded sentence: Salutt!

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Hi.
Decoded sentence: Salutt!

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Run!
Decoded sentence: Cunsentss-nous.

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Run!
Decoded sentence: Cunsentss-nous.

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Run!
Decoded sentence: Cunsentss-nous.

1/1 [==============================] - 0s 18ms/step
-
Input sent